In [3]:
import pandas as pd
import numpy as np
import datetime


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [ ]:
#table_name = 'YOUR_SNOWFLAKE_TABLE_NAME'

#sf_df = my_session.sql("select * from {}".format(table_name))
#df = sf_df.to_pandas()

In [4]:
df = pd.read_csv('Attendance_dataset.csv')

In [6]:
df.shape

(1000, 5)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Employee_Id      1000 non-null   object
 1   Employee_name    1000 non-null   object
 2   Attendance_Date  1000 non-null   object
 3   Present_Status   1000 non-null   object
 4   Hours_Worked     1000 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 39.2+ KB
